In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import os
import torch
from transformers import AutoModel, AutoTokenizer
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPool1D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
import time
import time
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
!pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 4.8 MB/s eta 0:00:00


In [ ]:
df = pd.read_excel("/content/drive/MyDrive/colab/behaviour_simulation_train.xlsx")

In [ ]:
tweet = []
for i in range(300000):
    tweet.append(df['content'][i] + " " + df['date'][i] + " " + df['username'][i] + " " + df['inferred company'][i])

In [ ]:
df['tweet'] = tweet

In [ ]:
df['likes'] = pd.to_numeric(df['likes'], errors='coerce')
df = df.dropna(subset=['likes'])

df['likes'] = df['likes'].astype(int)

In [ ]:
df['tweet'] = df['tweet'].astype(str)

In [ ]:
import torch
import numpy as np
import time
from transformers import AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False)
bertweet = AutoModel.from_pretrained("vinai/bertweet-base")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bertweet.to(device)
bertweet.eval()

batch_size = 64

def get_bertweet_embeddings(sentences, model, tokenizer, device):
    inputs = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)

    print("Input tensor shape:", inputs["input_ids"].shape)

    with torch.no_grad():
        outputs = model(**inputs)

    return outputs.last_hidden_state.mean(dim=1).cpu().numpy()

start = time.time()
print("train data embeddings started")

X_test = None

for start_idx in range(0, len(df), batch_size):
    end_idx = min(start_idx + batch_size, len(df))
    batch_sentences = [text for text in df["tweet"].iloc[start_idx:end_idx].tolist() if text]

    try:
        X_batch = get_bertweet_embeddings(batch_sentences, bertweet, tokenizer, device)
    except RuntimeError as e:
        print(f"Error processing batch {start_idx} to {end_idx}: {e}")
        continue

    if start_idx == 0:
        X_test = X_batch
    else:
        X_test = np.concatenate([X_test, X_batch])

np.save("content-test-company-full.npy", X_test)

print(f"Time required = {time.time() - start}")
print("test data embeddings ended")
